In [4]:
import pandas as pd
import numpy as np
import ast 
import json 

In [5]:
df1= pd.read_excel('Dataset/movies_dataset.xlsx')
#data1 = pd.read_csv('Dataset/movies_dataset.csv')

#data1 = pd.read_csv('Dataset/movies_dataset.csv, low_memory=False')


In [6]:
df2 = pd.read_csv('Dataset/credits.csv')

In [7]:
df1

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30 00:00:00,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,0.0,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15 00:00:00,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,0.0,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22 00:00:00,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,0.0,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22 00:00:00,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,0.0,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10 00:00:00,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,0.0,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,0.0,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17 00:00:00,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,0.0,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01 00:00:00,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,0.0,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21 00:00:00,0.0,87.0,[],Released,NaN,Satan Triumphant,0.0,0.0,0.0


In [8]:
#convertimos los id en enteros para evitar complicaciones
df1['id'] = pd.to_numeric(df1['id'], errors='coerce').astype('Int64')
df2['id'] = pd.to_numeric(df2['id'], errors='coerce').astype('Int64')

df1 = df1.dropna(subset=['id']) #eliminamos datos con id vacios
df2 = df2.dropna(subset=['id']) #eliminamos datos con id vacios

df1 = df1.drop_duplicates(subset='id')  # Eliminar filas duplicadas basadas en la columna 'id'
df2= df2.drop_duplicates(subset='id')  # Eliminar filas duplicadas basadas en la columna 'id'


In [9]:
# df_union = pd.concat([df1.drop('id', axis=1), df2], axis=1)


In [10]:
# uniomos las dos data en una sola
df_union = pd.concat([df1, df2], axis=1)


In [11]:
# eliminamos las columnas que no vayamos a necesitar para nuestros analisis aportados por el empleador
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df_union = df_union.drop(columns_to_drop, axis=1)

In [12]:
#Usamos  el método fillna() de panda, para llenar los campos de revenue" y "budget". 
df_union['revenue'] = df_union['revenue'].fillna(0)
df_union['budget'] = df_union['budget'].fillna(0)

In [13]:
#eliminamos los valores nulos de release_date

df_union['release_date'] = df_union['release_date'].fillna(np.nan)
df_union = df_union.dropna(subset=['release_date'])

In [14]:
#imprimimos los cambios de eliminacion de valores nulos.
print(df_union['release_date'].isnull().sum())


0


In [15]:
print(df_union.columns)


Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew', 'id'],
      dtype='object')


In [16]:
#cambiamos el formato de fecha de release_date y creamos una columna release_year en donde tendremos el año de estreno
df_union['release_date'] = pd.to_datetime(df_union['release_date']).dt.strftime('%Y-%m-%d')
df_union['release_year'] = pd.to_datetime(df_union['release_date']).dt.year

In [17]:
#cambiamos y verificamos el dtype de nuestros datos en release_date
df_union['release_date'] = pd.to_datetime(df_union['release_date'])

#print(df_union['release_date'].dtype)
#print(df_union['release_year'].dtype)

In [18]:
#Cambiamos los valores no finitos por nan
df_union['release_year'] = df_union['release_year'].replace([np.inf, -np.inf], np.nan)

In [19]:
#eliminamos los datos nulos o no finitos
df_union = df_union.dropna(subset=['release_year'])

In [20]:

#convertimos los datos a enteros
df_union['release_year'] = df_union['release_year'].astype(int)

In [21]:
#miramos los cambios
df_union['release_year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45346, dtype: int32

In [22]:
#eliminar las filas duplicadas y las filas con valores nulos en la columna "id" del DataFrame df_union

df_union = df_union.drop_duplicates(subset=['id'])
df_union = df_union.dropna(subset=['id'])

In [23]:
df_union

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,id,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,1995
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,1995
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,1995
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,1995
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,5.7,26.0,"[{'cast_id': 3, 'character': 'All the members ...","[{'credit_id': '52fe478dc3a36847f813bd6b', 'de...",49280,1991
45462,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0,[],"[{'credit_id': '539ef1090e0a263dd00000d7', 'de...",276895,2011
45463,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betraya

In [24]:
#Creamos una columna con el retorno de inversión, llamada return con los campos revenue / budget , en caso de no tener datos obtendremos cero.


def calculate_return(row):
    if pd.isnull(row['revenue']) or pd.isnull(row['budget']) or row['budget'] == 0:
        return 0
    else:
        return row['revenue'] / row['budget']

df_union['return'] = df_union.apply(calculate_return, axis=1)

## desanidamos Belong to conection

In [25]:
from pandas import json_normalize
import ast
import json
import numpy as np


In [26]:
#Desanidamos la columna Belong_to_conection y luego creamos columnas con esos datos
df_union['belongs_to_collection'] = df_union['belongs_to_collection'].apply(lambda x: {} if pd.isna(x) else eval(x))
df_collection = json_normalize(df_union['belongs_to_collection']).add_prefix('collection_')
df_union = pd.concat([df_union, df_collection], axis=1)

In [27]:
production_companies_data = df_union['production_companies']
print(production_companies_data)


0           [{'name': 'Pixar Animation Studios', 'id': 3}]
1        [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2        [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3        [{'name': 'Twentieth Century Fox Film Corporat...
4        [{'name': 'Sandollar Productions', 'id': 5842}...
                               ...                        
45070                                                  NaN
45148                                                  NaN
45203                                                  NaN
45265                                                  NaN
45275                                                  NaN
Name: production_companies, Length: 45463, dtype: object


In [28]:
df_union['collection_id'] = pd.to_numeric(df_union['collection_id'], errors='coerce', downcast='integer')


In [29]:
#eliminamos columna que ya no vamos a necesitar
df_union = df_union.drop(['belongs_to_collection'], axis=1)


In [30]:
#Renombramos la columna colecccollection_name, para poder usarlas de mejor forma en nuestra api
df_union = df_union.rename(columns={'collection_name': 'franchise'})


In [31]:
#reemplazamos los valores faltantes en la columna "franchise" del DataFrame df_union por una cadena vacía (" ")
df_union['franchise'] = df_union['franchise'].fillna(" ")

In [32]:
df_union

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,vote_count,cast,crew,id,release_year,return,collection_id,franchise,collection_poster_path,collection_backdrop_path
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,1995.0,12.451801,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,...,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,1995.0,4.043035,NaN,,NaN,NaN
2,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,...,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,1995.0,0.000000,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,...,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,1995.0,5.090760,NaN,,NaN,NaN
4,0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,...,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,1995.0,0.000000,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45070,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,,NaN,NaN
45148,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,,NaN,NaN
45203,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,,NaN,NaN
45265,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,,NaN,NaN


In [33]:
#eliminamos columnas que no necesitaremos del desanidado de la anterior columna
df_union = df_union.drop(['collection_poster_path','collection_backdrop_path'], axis=1)


In [34]:
df_union['production_companies']

0           [{'name': 'Pixar Animation Studios', 'id': 3}]
1        [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2        [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3        [{'name': 'Twentieth Century Fox Film Corporat...
4        [{'name': 'Sandollar Productions', 'id': 5842}...
                               ...                        
45070                                                  NaN
45148                                                  NaN
45203                                                  NaN
45265                                                  NaN
45275                                                  NaN
Name: production_companies, Length: 45463, dtype: object

# produccion companies

In [35]:
from pandas import json_normalize
import json


In [36]:
#Desanidar Production_copanies
df_union['production_companies'] = df_union['production_companies'].apply(lambda x: ', '.join([company['name'] for company in ast.literal_eval(x)]) if (pd.notna(x) and isinstance(x, str)) else '')


In [37]:
production_companies_dict = {}

In [38]:
# df_union['productores'] = [production_companies_dict.get(index, []) for index in df_union.index]


In [39]:
#df_union = df_union.drop(['cast'], axis=1)

In [40]:
df_union['production_companies']

0                                  Pixar Animation Studios
1        TriStar Pictures, Teitler Film, Interscope Com...
2                             Warner Bros., Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4               Sandollar Productions, Touchstone Pictures
                               ...                        
45070                                                     
45148                                                     
45203                                                     
45265                                                     
45275                                                     
Name: production_companies, Length: 45463, dtype: object

In [41]:
#DESANIDAR GENEROS
df_union['genres'] = df_union['genres'].apply(lambda x: [] if pd.isna(x) else ast.literal_eval(x))

In [42]:
df_union['generos'] = df_union['genres'].apply(lambda x: [genre['name'] for genre in x])

In [43]:
df_union[['generos']].head()



,generos
0,"[Animation, Comedy, Family]"
1,"[Adventure, Fantasy, Family]"
2,"[Romance, Comedy]"
3,"[Comedy, Drama, Romance]"
4,[Comedy]


In [44]:
#DESANIDAMOS LA COLUMNA PRODUCTION_COUNTRIES
df_union['production_countries'] = df_union['production_countries'].apply(lambda x: [] if pd.isna(x) else ast.literal_eval(x))

In [45]:
df_union['pais_production'] = df_union['production_countries'].apply(lambda x: [production_countries['name'] for production_countries in x])

In [46]:
df_union.rename(columns={'pais_production': 'production_countries02'}, inplace=True)

In [47]:
df_union = df_union.drop(['production_countries'], axis=1)


In [48]:
# Reemplazar valores NaN por listas vacías
df_union['crew'] = df_union['crew'].fillna('[]')

# Convertir las cadenas de texto en listas de diccionarios
df_union['crew'] = df_union['crew'].apply(ast.literal_eval)

# Extraer el nombre del director de la columna 'crew'
df_union['Director'] = df_union['crew'].apply(lambda crew: [member['name'] for member in crew if member['job'] == 'Director'][0] if any(member['job'] == 'Director' for member in crew) else '')

In [49]:
# Mostrar el DataFrame resultante
print(df_union['Director'])

0          John Lasseter
1           Joe Johnston
2          Howard Deutch
3        Forest Whitaker
4          Charles Shyer
              ...       
45070                   
45148                   
45203                   
45265                   
45275                   
Name: Director, Length: 45463, dtype: object


In [50]:
df_union

,budget,genres,id,original_language,overview,popularity,production_companies,release_date,revenue,runtime,...,cast,crew,id,release_year,return,collection_id,franchise,generos,production_countries02,Director
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,1995-10-30,373554033.0,81.0,...,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,1995.0,12.451801,10194.0,Toy Story Collection,"[Animation, Comedy, Family]",[United States of America],John Lasseter
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",1995-12-15,262797249.0,104.0,...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,1995.0,4.043035,NaN,,"[Adventure, Fantasy, Family]",[United States of America],Joe Johnston
2,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",1995-12-22,0.0,101.0,...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,1995.0,0.000000,119050.0,Grumpy Old Men Collection,"[Romance, Comedy]",[United States of America],Howard Deutch
3,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,1995-12-22,81452156.0,127.0,...,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,1995.0,5.090760,NaN,,"[Comedy, Drama, Romance]",[United States of America],Forest Whitaker
4,0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",1995-02-10,76578911.0,106.0,...,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,1995.0,0.000000,96871.0,Father of the Bride Collection,[Comedy],[United States of America],Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45070,NaN,[],<NA>,NaN,NaN,NaN,,NaT,NaN,NaN,...,NaN,[],<NA>,NaN,NaN,NaN,,[],[],
45148,NaN,[],<NA>,NaN,NaN,NaN,,NaT,NaN,NaN,...,NaN,[],<NA>,NaN,NaN,NaN,,[],[],
45203,NaN,[],<NA>,NaN,NaN,NaN,,NaT,NaN,NaN,...,NaN,[],<NA>,NaN,NaN,NaN,,[],[],
45265,NaN,[],<NA>,NaN,NaN,NaN,,NaT,NaN,NaN,...,NaN,[],<NA>,NaN,NaN,NaN,,[],[],


In [51]:
# Contar filas sin director
num_rows_without_director = len(df_union[df_union['Director'] == ''])

# Eliminar filas sin director
df_union = df_union[df_union['Director'] != '']

# Restablecer los índices del DataFrame
df_union = df_union.reset_index(drop=True)

In [52]:
# Mostrar el número de filas eliminadas
print(f"Número de filas eliminadas: {num_rows_without_director}")

# Mostrar el DataFrame resultante
print(df_union[['crew', 'Director']])

Número de filas eliminadas: 1027
                                                    crew         Director
0      [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    John Lasseter
1      [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...     Joe Johnston
2      [{'credit_id': '52fe466a9251416c75077a89', 'de...    Howard Deutch
3      [{'credit_id': '52fe44779251416c91011acb', 'de...  Forest Whitaker
4      [{'credit_id': '52fe44959251416c75039ed7', 'de...    Charles Shyer
...                                                  ...              ...
44431  [{'credit_id': '52fe478dc3a36847f813bd6b', 'de...   Georges Méliès
44432  [{'credit_id': '539ef1090e0a263dd00000d7', 'de...   Robert Gardner
44433  [{'credit_id': '58ee55bbc3a3683df500bd0f', 'de...     Ravi Udyawar
44434  [{'credit_id': '587626f4c3a3682b33008299', 'de...   Shanra J. Kehl
44435  [{'credit_id': '5330178a9251413a720055e3', 'de...    Paolo Taviani

[44436 rows x 2 columns]


##FUNCIONES Y DESARROLLO DE API

In [53]:
from fastapi import FastAPI #Importamos las librerias necesarias para el desarrollo de la api


In [56]:
# Crea una instancia de FastAPI
app = FastAPI()

In [59]:
#Probamos las distintas funciones de la API antes de usarlas en archivo main.py, para evitar errores de sintaxis

# Endpoint para obtener la cantidad de películas producidas en un idioma específico y su relación con el año de estreno
@app.get('/peliculas_idioma{idioma}')
def peliculas_idioma(idioma: str):
    filtered_df = df_union[df_union['original_language'] == idioma]
    count = filtered_df.shape[0]
    years = filtered_df['release_year'].tolist()
    return {"idioma": idioma, "cantidad": count, "años_estreno": years}

In [60]:
# Endpoint para obtener la duración y el año de una película específica
@app.get('/peliculas_duracion/{pelicula}')
def peliculas_duracion(pelicula: str):
    peli=df_union[df_union['title']== pelicula].iloc[0]
    duracion=peli['runtime']
    anio=peli['release_year']
    return{'respuesta':f"(pelicula),duracion:{duracion}, Año:{anio}"}
                                                    

In [64]:
# Endpoint para obtener información de una franquicia específica
@app.get('/franquicia')
def franquicia(Franquicia: str):
    filtered_df = df_union[df_union['franchise'] == Franquicia]
    if filtered_df.empty:
        return {"mensaje": f"No se encontró la franquicia '{Franquicia}'."}
    cantidad_peliculas = filtered_df.shape[0]
    ganancia_total = filtered_df['revenue'].sum()
    ganancia_promedio = filtered_df['revenue'].mean()
    return {"Franquicia": Franquicia, 
            "Cantidad de películas": cantidad_peliculas, 
            "Ganancia total": ganancia_total, 
            "Ganancia promedio": ganancia_promedio}

In [63]:
# Endpoint para obtener la cantidad de películas producidas en un país específico
@app.get('/peliculas_pais/{pais}')
def peliculas_pais(pais: str):
    filtered_df = df_union[df_union['production_countries02'].str.contains(pais, case=False)]
    cantidad_peliculas = filtered_df.shape[0]
    return {"mensaje": f"Se produjeron {cantidad_peliculas} películas en el país {pais}"}

In [62]:
# Endpoint para obtener información de una productora específica
@app.get('/productoras_exitosas/{productora}')
def productoras_exitosas(productora: str):
    df_cleaned = df_union.dropna(subset=['production_companies'])
    filtered_df = df_cleaned[df_cleaned['production_companies'].str.contains(productora, case=False)]
    count = filtered_df.shape[0]
    return {"productora": productora, "cantidad": count}

In [61]:
# Endpoint para obtener información sobre un director específico
@app.get('/get_director')
def get_director(nombre_director: str):
    filtered_df = df_union[df_union['Director'].str.contains(nombre_director, case=False)]
    peliculas = []
    
    for _, row in filtered_df.iterrows():
        pelicula = {
            "nombre": row['title'],
            "fecha_lanzamiento": str(row['release_date']),
            "retorno_individual": row['return'],
            "costo": row['budget'],
            "ganancia": row['revenue']
        }
        peliculas.append(pelicula)
    
    return {"director": nombre_director, "peliculas": peliculas}

In [ ]:
#se usara el archivo main.py para realizar las distintas funciones de la API

In [65]:
#exportamos la data para poder usarla en nuestro archivo main.py
df_union.to_excel('df_union.xlsx', index=False)
